In [59]:
    import pygments.formatters.other, IPython, html, nbconvert

In [60]:
    def format_title(object): return '&#013;'.join(html.escape(nbconvert.filters.strip_ansi(str(object))).splitlines())

In [76]:
    class EscapedFormatter:
        def format(self, tokensource, outfile):
            _escape_html_table = pygments.formatters.html._escape_html_table 
            pygments.formatters.html._escape_html_table = {}
            try: return super().format(tokensource, outfile)
            finally: pygments.formatters.html._escape_html_table = _escape_html_table

In [84]:
    class NamingLexer(pygments.lexers.PythonLexer):
        def get_tokens(self, text, unfiltered=False):
            buffer_token, buffer = None, "''"
            for token, snippet in list(super().get_tokens(text)) + [
                (pygments.token.Token.Generic, '')
            ]:
                if buffer and ((snippet == '.') or buffer.endswith('.')):  
                    buffer += snippet
                elif token in {
                    pygments.token.Token.Name, pygments.token.Token.Name.Namespace, 
                    pygments.token.Token.Name.Class, pygments.token.Token.Name.Function
                }:
                    buffer_token, buffer = token, snippet            
                else:
                    if buffer_token and buffer: 
                        try: 
                            title = IPython.get_ipython().object_inspect_text(
                                buffer, detail_level=buffer_token in {pygments.token.Token.Name.Function}) 
                            yield buffer_token, F"""<span style="{title and "border-bottom: 1px dotted gray;" or ""
                            }" title="{format_title(title)}">{buffer}</span>"""
                        except: 
                            yield buffer_token, buffer
                        buffer_token, buffer = None, ""
                    yield token, snippet

In [87]:
    def markup_code(source, format=pygments.formatters.other.NullFormatter):
        """
        <style>{{pygments.formatters.HtmlFormatter().get_style_defs()}}</style>        
        """
        return pygments.highlight(source, NamingLexer(), type('formatter', (EscapedFormatter, format), {})())

In [90]:
    def markup_code_inline(source):
        """
        <style>{{pygments.formatters.HtmlFormatter().get_style_defs()}}</style>        
        """
        inline
        return (markup_code(source))

In [89]:
    def inline(html):
        lead, sep, html = html.partition('div')
        html = lead + 'span' + html
        lead, sep, html = html.rpartition('/div')
        html = lead + '/span' + html
        lead, sep, html = html.partition('<pre>')
        html = lead + html
        lead, sep, html = html.partition('</pre>')
        return lead + html